In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
import scipy
import scipy.sparse as sp
import json

import os
import pickle
import json

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from bs4 import BeautifulSoup

from utils import weatherapi
from utils import isw_preprocessing
from utils import text_preprocessing

from sklearn import preprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dusty3ntity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dusty3ntity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dusty3ntity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
MODELS_FOLDER = "data/models"
OUTPUT_FOLDER = "data/prediction"

if not os.path.exists(OUTPUT_FOLDER):
   os.makedirs(OUTPUT_FOLDER)

ISW_DATA_FILE = "data/isw/reports_processed_v2.csv"
WEATHER_EVENTS_DATA_FILE = "data/alarms/merged_events.csv"

tfidf_transformer_model = "tfidf_transformer"
tfidf_transformer_version = "v1"

count_vectorizer_model = "count_vectorizer"
count_vectorizer_version = "v1"

decision_tree_model = "DTC"
logistic_regression_model = "LR"
multi_layer_perceptron_model = "MLP"
random_forest_classifier_model = "RFC"
sgd_classifier_model = "SGD"

## 1. Load models & data

In [15]:
tfidf = pickle.load(open(f"data/isw/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"data/isw/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

label_encoder = pickle.load(open(f"data/isw/hour_conditions_label_encoder.pkl", "rb"))

DTC = pickle.load(open(f"{MODELS_FOLDER}/{decision_tree_model}.pkl", "rb"))
LR = pickle.load(open(f"{MODELS_FOLDER}/{logistic_regression_model}.pkl", "rb"))
MLP = pickle.load(open(f"{MODELS_FOLDER}/{multi_layer_perceptron_model}.pkl", "rb"))
RFC = pickle.load(open(f"{MODELS_FOLDER}/{random_forest_classifier_model}.pkl", "rb"))
SGD = pickle.load(open(f"{MODELS_FOLDER}/{sgd_classifier_model}.pkl", "rb"))

In [16]:
REGIONS_DICTIONARY_FILE = "data/data_before_lab_3/regions.csv"
df_regions = pd.read_csv(REGIONS_DICTIONARY_FILE)
df_regions.head(25)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5
5,Житомирська,Житомир,Zhytomyr,Житомирщина,6
6,Закарпатська,Ужгород,Uzhgorod,Закарпаття,7
7,Запорізька,Запоріжжя,Zaporozhye,Запоріжжя,8
8,Івано-Франківська,Івано-Франківськ,Ivano-Frankivsk,Івано-Франківщина,9
9,Київська,Київ,Kyiv,Київщина,10


## 2. Setup prediction region

In [17]:
region = "Львівщина"
date = "2023-04-08"

city = df_regions[df_regions["region_alt"]==region]["center_city_en"].values[0]

## 3. Get weather

In [18]:
level = "hours"
location = f"{city},Ukraine"

file_name = f"weather___{city.lower()}__{date}.json"

In [21]:
if not os.path.isfile(f"{OUTPUT_FOLDER}/{file_name}"):
	city_weather_json = weatherapi.get_weather_json(location, date)
	json_object = json.dumps(city_weather_json, indent=4)

	with open(f"{OUTPUT_FOLDER}/{file_name}", "w") as outfile:
		outfile.write(json_object)
else:
	print(f"Weather data from the \nregion {region}; \ndate {date}; \n is ready")

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Lviv,Ukraine/1681124568/1681167768?key=3QULGS8KJPBECC4J3W7MPEF2T


In [22]:
# weather_file_path = f"{OUTPUT_FOLDER}/{file_name}"
# weather_for_day_hourly = weather.get_weather_hourly_for_region(weather_file_path)
# weather_df = pd.DataFrame.from_dict(weather_for_day_hourly)

# Getting random weather from the dataset as the API is not ready yet
all_weather_by_hour = pd.read_csv("data/data_before_lab_3/all_weather_by_hour.csv")
weather_df = all_weather_by_hour.loc[all_weather_by_hour['day_datetime'] == "2022-04-08"]

weather_df["day_datetime"] = pd.to_datetime(weather_df["day_datetime"])
weather_df["city"] = weather_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
weather_df["city"] = weather_df["city"].replace("Хмельницька область", "Хмельницький")

C:\Users\dusty3ntity\AppData\Local\Temp\ipykernel_32896\536417939.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df["day_datetime"] = pd.to_datetime(weather_df["day_datetime"])
C:\Users\dusty3ntity\AppData\Local\Temp\ipykernel_32896\536417939.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df["city"] = weather_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
C:\Users\dusty3ntity\AppData\Local\Temp\ipykernel_32896\536417939.py:11: SettingWithCopyWarning: 
A value is tryin

In [36]:
weather_df_v2 = pd.merge(weather_df, df_regions, left_on="city", right_on="center_city_ua")

## 3. Get ISW report

In [3]:
date = "2023-04-08"

BASE_URL = "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment"
months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
date_arr = date.split("-")

url = f"{BASE_URL}-{months[int(date_arr[1])-1]}-{int(date_arr[2])}-{int(date_arr[0])}"
print(url)

page = requests.get(url)
isw_report_html_file = f"{OUTPUT_FOLDER}/isw___{date}.html"
with open(isw_report_html_file, "wb+") as f:
  f.write(page.content)

https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-8-2023


In [6]:
with open(isw_report_html_file, "r", encoding="utf8") as cfile:
		html = BeautifulSoup(cfile.read(), features="html.parser")
		content_html = str(html.body.find("div", attrs={"class": "field-type-text-with-summary"}))

		content_html = isw_preprocessing.preprocess_page_html(content_html)
		content_text_lemm = text_preprocessing.text_preprocess(content_html, "lemm")

In [9]:
content_text_lemm_list = content_text_lemm.split(" ")
cv = CountVectorizer(max_df=0.98, min_df=2)
word_count_vector = cv.fit_transform(content_text_lemm_list)

In [29]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)
tfidf_vector = tfidf_transformer.transform(word_count_vector)

## 4. Data preparation

In [46]:
weather_exclude = [
"day_feelslikemax",
"day_feelslikemin",
"day_sunriseEpoch",
"day_sunsetEpoch",
"day_description",
"city_latitude",
"city_longitude",
"city_address",
"city_timezone",
"city_tzoffset",
"day_feelslike",
"day_precipprob",
"day_snow",
"day_snowdepth",
"day_windgust",
"day_windspeed",
"day_winddir",
"day_pressure",
"day_cloudcover",
"day_visibility",
"day_severerisk",
"day_conditions",
"day_icon",
"day_source",
"day_preciptype",
"day_stations",
"hour_icon",
"hour_source",
"hour_stations",
"hour_feelslike"
]

fields_to_exclude = [
	"city_resolvedAddress",
	"day_datetime",
	"day_datetimeEpoch",
	"hour_datetime",
	"hour_datetimeEpoch",
	"city",
	"region",
	"center_city_ua",
	"center_city_en"
]

tmp_fields_to_exclude = [
	"day_sunrise",
	"day_sunset",
	"hour_preciptype",
	"hour_conditions",
	"hour_solarenergy",
	"region_alt"
]

df_work_v2 = weather_df_v2.drop(weather_exclude, axis=1)
df_work_v2 = df_work_v2.drop(fields_to_exclude, axis=1)

In [47]:
df_work_v2["hour_conditions"] = df_work_v2["hour_conditions"].apply(lambda x: x.split(",")[0])
df_work_v2["hour_conditions_id"] = label_encoder.transform(df_work_v2["hour_conditions"])

In [48]:
df_work_v3 = df_work_v2.drop(tmp_fields_to_exclude, axis=1)

In [49]:
df_work_v3.head(10)

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,...,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,region_id,hour_conditions_id
0,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,9.7,157.8,1002.0,24.1,0.0,0.0,0.0,10.0,23,0
1,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,6.5,162.9,1002.0,24.1,11.5,0.0,0.0,10.0,23,0
2,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,4.0,126.6,1001.0,24.1,47.6,0.0,0.0,10.0,23,4
3,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,4.3,111.8,1001.0,24.1,86.3,0.0,0.0,10.0,23,4
4,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,5.0,97.7,1001.0,24.1,64.2,0.0,0.0,10.0,23,4
5,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,6.5,104.9,1000.0,24.1,38.3,0.0,0.0,10.0,23,4
6,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,7.6,107.8,999.0,24.1,100.0,0.0,0.0,10.0,23,3
7,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,7.6,120.6,999.0,24.1,82.3,14.0,0.0,10.0,23,4
8,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,7.9,124.2,999.0,24.1,100.0,90.0,1.0,10.0,23,3
9,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,...,7.2,140.7,998.0,24.1,91.4,176.0,2.0,10.0,23,3


In [43]:
tfidf_matrix = tfidf_vector

for i in range(0, 23):
	tfidf_matrix = sp.vstack((tfidf_matrix, tfidf_vector), format="csr")

In [51]:
df_work_v4_csr = sp.csr_matrix(df_work_v3.values)
df_all_features = sp.hstack((df_work_v4_csr, tfidf_matrix), format="csr")

ValueError: Mismatching dimensions along axis 0: {576, 77904}

In [ ]:
hours_alarm_schedule = DTC.predict(df_all_features)

In [ ]:
hours_alarm_schedule